In [3]:
!pip3 install faker

  Using cached faker-37.4.0-py3-none-any.whl.metadata (15 kB)
Using cached faker-37.4.0-py3-none-any.whl (1.9 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Creating a Dummy HealthCare Dataset to work on real-industrial level case studies for Data Analyst

''Before running the below cell, Create a dummy_healthcare.csv file in your directory''

In [4]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker()
np.random.seed(42)
random.seed(42)

n_rows = 100_000

data = {
    "Patient_ID": [f"PID{100000 + i}" for i in range(n_rows)],
    "Name": [fake.name() for _ in range(n_rows)],
    "Age": np.random.randint(0, 100, size=n_rows),
    "Gender": np.random.choice(["Male", "Female", "Other"], size=n_rows),
    "Country": np.random.choice(["USA", "India", "UK", "Canada", "Germany"], size=n_rows, p=[0.4, 0.3, 0.1, 0.1, 0.1]),
    "Disease": np.random.choice(
        ["Diabetes", "Hypertension", "Cancer", "Heart Disease", "COVID-19", "None"],
        size=n_rows,
        p=[0.2, 0.25, 0.15, 0.1, 0.05, 0.25]
    ),
    "Admission_Date": [fake.date_between(start_date="-3y", end_date="today") for _ in range(n_rows)],
    "Cost_USD": np.round(np.abs(np.random.normal(loc=3000, scale=1000, size=n_rows)), 2)
}

health_df = pd.DataFrame(data)
health_df.to_csv("dummy_healthcare_data.csv", index=False)
print("✅ Dataset saved as dummy_healthcare_data.csv")


✅ Dataset saved as dummy_healthcare_data.csv


DATA ANALYSIS USING PYTHON(PANDAS)

In [6]:
df = pd.read_csv("C:/Users/rakes/OneDrive/Desktop/HealthCare/dummy_healthcare_data.csv")
df.head()

,Patient_ID,Name,Age,Gender,Country,Disease,Admission_Date,Cost_USD
0,PID100000,Daniel Martin,51,Other,India,Cancer,2022-11-10,2463.91
1,PID100001,Monica Ballard,92,Male,India,Hypertension,2023-08-05,1059.32
2,PID100002,Julie Walsh,14,Other,Germany,None,2025-02-07,1901.73
3,PID100003,Adam King,71,Other,India,Hypertension,2023-01-02,936.05
4,PID100004,Natalie Hamilton,60,Female,India,COVID-19,2024-04-07,3237.85


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Patient_ID      100000 non-null  object 
 1   Name            100000 non-null  object 
 2   Age             100000 non-null  int64  
 3   Gender          100000 non-null  object 
 4   Country         100000 non-null  object 
 5   Disease         100000 non-null  object 
 6   Admission_Date  100000 non-null  object 
 7   Cost_USD        100000 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 6.1+ MB


In [8]:
df.describe()

,Age,Cost_USD
count,100000.000000,100000.000000
mean,49.448240,3000.079171
std,28.950724,998.349551
min,0.000000,3.380000
25%,24.000000,2323.897500
50%,49.000000,2998.745000
75%,75.000000,3674.570000
max,99.000000,7835.120000


In [ ]:
# CHECKING FOR NULL VALUES
df.isna().sum()

Patient_ID        0
Name              0
Age               0
Gender            0
Country           0
Disease           0
Admission_Date    0
Cost_USD          0
dtype: int64

Converting the Datatypes into required format

In [35]:
df['Admission_Date'] = pd.to_datetime(df['Admission_Date'])

1. “Can you tell me which 3 countries have the highest number of patients in the last 1 year?”

→ Keywords: filter by Admission_Date, group by Country, count patients
Goal: Understand geographic demand trends

In [ ]:
from datetime import datetime,timedelta

last_df = df[['Country','Patient_ID','Admission_Date']]

today_date = datetime.now() - timedelta(days=365)

cutoff = last_df['Admission_Date'].max() - timedelta(days=365)

recent_df = last_df[last_df['Admission_Date']>=cutoff]

top_3 = recent_df['Country'].value_counts().head(3)
print("Countries with highest no of patients in last year",'\n',top_3)

Countries with highest no of patients in last year 
 USA      13530
India    10032
UK        3390
Name: Country, dtype: int64


In [143]:
# today_date = datetime.now().year - 2
today_date = datetime.now() - timedelta(days=365)
today_date

datetime.datetime(2024, 7, 11, 11, 48, 23, 204780)

2. “I’d like to know the average cost per disease across all countries. Can you compare them?”

→ Keywords: groupby, pivot_table, mean, Cost_USD, Disease
Goal: Financial analysis per illness

In [164]:
diseases_df = df[['Patient_ID',"Country","Disease","Cost_USD"]]

# avg_cost_per_diesease = diseases_df.groupby('Disease')['Cost_USD'].agg('mean')

avg_cost_per_diesease = diseases_df.groupby('Disease').agg({'Cost_USD':"mean"}).round(2).sort_values(by='Cost_USD').rename(columns={'Cost_USD': "Avg_Cost_per_dieases($)"})

avg_cost_per_diesease 

,Avg_Cost_per_dieases($)
Disease,
Heart Disease,2993.61
Hypertension,2996.33
Cancer,2997.79
COVID-19,3000.62
None,3002.18
Diabetes,3006.86


3. “We’re concerned about elderly patients (Age > 60). How many of them were admitted for each disease?”

→ Keywords: filter on Age, group by Disease, count
Goal: Age-risk segmentation

In [ ]:
age_df = df[['Patient_ID','Age','Disease']]

age = age_df[df['Age']>60]

age_group = age.groupby('Disease').agg({'Age':"count"})

age_group

,Age
Disease,
COVID-19,2012
Cancer,5740
Diabetes,7799
Heart Disease,3819
Hypertension,9635
None,9878


4. “Give me the trend of total healthcare cost month-by-month for the past 2 years.”

→ Keywords: datetime extraction (.dt.month, .dt.year), groupby, sum
Goal: Time series financial reporting

In [147]:
trend = df[['Admission_Date','Cost_USD']]

cutoff = trend['Admission_Date'].max()-timedelta(days=730)

month = trend[trend['Admission_Date']>=cutoff].copy()

month['YearMonth'] = month['Admission_Date'].dt.to_period('M')
month_by_month_trend = month.groupby('YearMonth')['Cost_USD'].agg("sum")
# month.head()
month_by_month_trend

YearMonth
2023-07    6172608.75
2023-08    8447586.42
2023-09    8352666.55
2023-10    8357702.84
2023-11    8023479.79
2023-12    8606064.36
2024-01    8181419.84
2024-02    7989412.24
2024-03    8713421.04
2024-04    8146211.89
2024-05    8336125.41
2024-06    8034080.71
2024-07    8454454.56
2024-08    8630635.11
2024-09    8228325.81
2024-10    8602084.15
2024-11    8244534.63
2024-12    8536821.20
2025-01    8719630.98
2025-02    7588376.31
2025-03    8340912.90
2025-04    8280348.37
2025-05    8525937.83
2025-06    8150845.02
2025-07    2666768.94
Freq: M, Name: Cost_USD, dtype: float64

5. “Can you list the top 5 most expensive patient cases — with patient name, disease, and cost?”

→ Keywords: sort_values, head()
Goal: Outlier / fraud investigation

In [ ]:
expense_patient = df[['Patient_ID','Name','Disease','Cost_USD']]

Top_5_expensive_customers = expense_patient.sort_values(by='Cost_USD',ascending=False)

Top_5_expensive_customers.head(5).reset_index()

,index,Patient_ID,Name,Disease,Cost_USD
0,61650,PID161650,Hannah Mitchell,Hypertension,7835.12
1,63589,PID163589,Molly Maxwell,None,7224.20
2,29327,PID129327,George Smith,COVID-19,7101.24
3,81701,PID181701,Michelle Roy,Diabetes,7007.50
4,43880,PID143880,Kelly Paul,None,6906.25


6. “What’s the average cost difference between male and female patients across different diseases?”

→ Keywords: groupby on Gender and Disease, aggregation on Cost_USD
Goal: Demographic cost analysis

In [151]:
gender_df = df[['Gender',"Cost_USD",'Disease']]
# gender_df.groupby(['Gender','Disease']).agg({'Cost_USD':'mean'}).round(2)
Gender_Comparison = (
    gender_df
    .groupby(['Gender','Disease'])
    .mean()
    .reset_index()
    .rename(columns={'Cost_USD':'Avg_cost_USD'})
)
Gender_Comparison

,Gender,Disease,Avg_cost_USD
0,Female,COVID-19,3007.016095
1,Female,Cancer,2999.460239
2,Female,Diabetes,2995.356635
3,Female,Heart Disease,2972.553530
4,Female,Hypertension,2994.563171
5,Female,None,3010.410770
6,Male,COVID-19,3016.480917
7,Male,Cancer,3002.226875
8,Male,Diabetes,3011.166841
9,Male,Heart Disease,3008.960726


7. “Build me a risk score: Patients older than 65 with serious diseases (Heart Disease, Cancer) should be labeled ‘High Risk’. Others can be ‘Low Risk’.”

→ Keywords: np.where() or apply(lambda)
Goal: Risk profiling

In [ ]:
new_df = df[['Name','Age','Disease']]
# new_df['Label'] = np.nan  #to create new column with empty values

# new_df['Label'] = np.where((new_df['Age']>65) &( new_df['Disease'].isin(['Heart Disease','Cancer'])),'High Risk','Low Risk')

def risk_laber(row):
    if row['Age'] >65 and row['Disease'] in ['Heart Disease','Cancer']:
        return 'High Risk'
    return 'Low_Risk'
new_df['Label'] = new_df.apply(risk_laber,axis=1)
new_df.head(10)

C:\Users\rakes\AppData\Local\Temp\ipykernel_27184\2792704175.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Label'] = new_df.apply(risk_laber,axis=1)


,Name,Age,Disease,Label
0,Daniel Martin,51,Cancer,Low_Risk
1,Monica Ballard,92,Hypertension,Low_Risk
2,Julie Walsh,14,None,Low_Risk
3,Adam King,71,Hypertension,Low_Risk
4,Natalie Hamilton,60,COVID-19,Low_Risk
5,Thomas Reynolds,20,None,Low_Risk
6,Tammy Martin,82,Heart Disease,High Risk
7,Matthew Franklin,86,Diabetes,Low_Risk
8,Christopher Johnson,74,Heart Disease,High Risk
9,Sarah Matthews,74,Hypertension,Low_Risk


8. “Show me the top 3 diseases by total treatment cost for each country.”

→ Keywords: groupby(['Country', 'Disease']), sum, nlargest() or custom sorting
Goal: Cost drivers by geography

In [ ]:
treatment = df[['Disease','Cost_USD','Country']]

grouped = treatment.groupby(['Country','Disease']).agg({'Cost_USD':'sum'})

grouped['Rank'] = grouped.groupby('Country').rank(method='first',ascending=False)

top_3_per_country = grouped[grouped['Rank']<=3].sort_values(['Country',"Rank"])
top_3_per_country

Cost_USD  Rank
Country Disease                        
Canada  None           7813734.76   1.0
        Hypertension   7469475.41   2.0
        Diabetes       6114852.86   3.0
Germany Hypertension   7619364.93   1.0
        None           7488386.20   2.0
        Diabetes       5846874.04   3.0
India   Hypertension  22235629.47   1.0
        None          22155467.36   2.0
        Diabetes      18055223.83   3.0
UK      None           7685239.21   1.0
        Hypertension   7466296.98   2.0
        Diabetes       6146391.72   3.0
USA     None          30082724.41   1.0
        Hypertension  29910853.83   2.0
        Diabetes      24268586.74   3.0

9. “I want to know if our billing is consistent. What’s the standard deviation of treatment cost per disease?”

→ Keywords: groupby('Disease'), std()
Goal: Cost consistency check

In [161]:
treat = df[['Disease','Cost_USD']]
treat.groupby('Disease').std().round(2).rename(columns={'Cost_USD':'STD'})


,STD
Disease,
COVID-19,1008.04
Cancer,995.61
Diabetes,996.10
Heart Disease,1001.24
Hypertension,1001.48
None,995.63


10. “Identify any patients with unusually high cost (greater than 3 standard deviations from the mean). Give their Patient_ID and disease.”

→ Keywords: zscore, np.abs() > 3, anomaly detection
Goal: Outlier detection

In [134]:
from scipy.stats import zscore
import numpy as np

health = df[['Patient_ID','Disease','Cost_USD']]

health['Zscore'] = zscore(health['Cost_USD'])

health[np.abs(health['Zscore'])>3].sort_values(by='Zscore',ascending=False).reset_index()

C:\Users\rakes\AppData\Local\Temp\ipykernel_27184\3499366723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health['Zscore'] = zscore(health['Cost_USD'])


,index,Patient_ID,Disease,Cost_USD,Zscore
0,61650,PID161650,Hypertension,7835.12,4.843058
1,63589,PID163589,None,7224.20,4.231125
2,29327,PID129327,COVID-19,7101.24,4.107961
3,81701,PID181701,Diabetes,7007.50,4.014066
4,43880,PID143880,None,6906.25,3.912648
...,...,...,...,...,...
129,95877,PID195877,Diabetes,6004.11,3.009012
130,25102,PID125102,Cancer,6003.03,3.007930
131,33411,PID133411,None,5999.32,3.004214
132,12100,PID112100,COVID-19,5997.19,3.002081
